# CHPT IV - EVALUATION DES MODELES

Objectif : trois techniques pour évaluer les modèles afin d'optimiser leur trade off biais/variance
- avoir un modèles qui minimise/maximise un score
- sur les données de test
- sans les avoir vue au préalable

Trois techniques :
- simple hold-out validation = un training set splitté en training/validation et un test qu'on n'utilisera seulement avant la mise en production
- shuffle hold-out avec iteration = on shuffle les données d'entrainement : on splitte train/validation on calcul l'erreur et on recommence...
- KFOLD = dans le cas de petit jeux de données ou simplement dans le cas d'une variance forte dans le premier cas : on divise notre jeux de données en K partie de même taille, chaque partie sera prise comme jeux de données de validation et d'entrainement, les scores pourront alors être moyennée
- KFOLD + shuffle iteration = les deux dernière technique, très bien pour les petits jeux de données mais très couteux en temps de calcul...

D'autres techniques peuvent être envisager, notamment utiliser des échantillons bootstrap dans le shuffling dans le cas de très petit jeux de données...



## Préparation des données et du modèles sur les données imdb

In [1]:
from keras.datasets import imdb
from keras import models, layers
import numpy as np

#ouverture
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=10000)

#prépa des données train et test
def vectorize_data(data,dim=10000):
    results = np.zeros((len(data),dim))
    for i, word in enumerate(data):
        results[i,word] = 1
    return results

x_train = vectorize_data(train_data)
x_test = vectorize_data(test_data)

y_train = np.asarray(train_labels).astype(float)
y_test = np.asarray(test_labels).astype(float)

# réseaux de neurones

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(16,activation = "relu",input_shape = (10000,)))
    model.add(layers.Dense(16,activation = "relu"))
    model.add(layers.Dense(1, activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\marec\miniconda3\envs\deeplwithpython\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\marec\miniconda3\envs\deeplwithpython\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different le

## Simple hold-out validation

Ce protocole d'apprentissage marche bien, le seul problème est qu'il faut suffisamment de données pour avoir trois jeux de données représentatif avec un jeux de données train suffisant pour l'apprentissage. On peut identifier des problèmes si on observe de grand changement de prédiction entre chaque shuffling des données, à ce moment là il est nécessaire de passer à un système KFOLD avec ou sans randomisation.

In [8]:

# important de mélanger le jeux de données dans le cas où il est n'est pas séquentielle (genre données temporelles quoi) avec np.random.shuffle()
# problème c'est que ca mélange aléatoirement les ligne de l'array mais ici on a train et test donc on va utiliser les indices plutot

indice_train = np.random.choice(np.arange(0,len(x_train)),len(x_train))

x_train = x_train[indice_train]
y_train = y_train[indice_train]

# on coupe en train validation
p = 0.75
l = len(x_train)
size = round(p*l)

x_ent = x_train[:size]
y_ent = y_train[:size]

x_val = x_train[size:]
y_val = y_train[size:]

model = build_model()

model.fit(x_ent,
          y_ent,
          epochs=5,
          batch_size=512,
         validation_data=(x_val,y_val))

model.evaluate(x_val,y_val)
## et la on continue pour améliorer notre modèle en revenant sur les hyperparamètres: la capacité (nombre de couche,nombre d'unité), le nombre d'epoch, le batch_size, la régularisation, le dropout, le type de couche
## ensuite, quand on est sur que notre modèle est optimisé en terme de précision et de généralisation on entraine sur tout le jeux de données et on test sur le jeux de donnée de test

model = build_model()

model.fit(x_train,y_train, epochs = 5, batch_size=512)

model.evaluate(x_test,y_test)

Epoch 1/5
37/37 [==============================] - 1s 25ms/step - loss: 0.4328 - accuracy: 0.8335 - val_loss: 0.2746 - val_accuracy: 0.9256
Epoch 2/5
37/37 [==============================] - 1s 18ms/step - loss: 0.1938 - accuracy: 0.9508 - val_loss: 0.1661 - val_accuracy: 0.9506
Epoch 3/5
37/37 [==============================] - 1s 18ms/step - loss: 0.1079 - accuracy: 0.9727 - val_loss: 0.1316 - val_accuracy: 0.9565
Epoch 4/5
37/37 [==============================] - 1s 18ms/step - loss: 0.0661 - accuracy: 0.9854 - val_loss: 0.0968 - val_accuracy: 0.9706
Epoch 5/5
196/196 [==============================] - 0s 1ms/step - loss: 0.0911 - accuracy: 0.9717
Epoch 1/5
49/49 [==============================] - 1s 14ms/step - loss: 0.3914 - accuracy: 0.8662
Epoch 2/5
49/49 [==============================] - 1s 14ms/step - loss: 0.1529 - accuracy: 0.9605
Epoch 3/5
49/49 [==============================] - 1s 15ms/step - loss: 0.0829 - accuracy: 0.9798
Epoch 4/5
49/49 [==============================

[0.45751768350601196, 0.8614400029182434]

## Validation croisée à K fold

L'objectif est de couper le jeux de données d'entrainement en K morceaux de même taille qui deviendront tour à tour jeux de données de test (un morceau) vs tous les autres morceaux en entrainement. L'avantage de ce type d'approche et d'obtenir des scores moyens et donc avoir une idée plus précise de la gé
néralisation dans le cas de jeux de données moyen/petit ainsi qu'avoir une idée de la variance associée au modèle. Un peu plus couteux en temps de calcul puisqu'il faudra faire tourner le modèle K fois.
On peut coupler cette technique avec un hold out si la taille du jeux de données le permet c'est toujours mieux

In [13]:
## Shuffling des données d'entrainement
indice = np.random.choice(np.arange(0,len(x_train)),len(x_train))
x_train = x_train[indice]
y_train = y_train[indice]

## Hold out pour la validation
p = 0.75
l = len(x_train)
size = round(0.75*l)

x_ent = x_train[:size]
y_ent = y_train[:size]
x_val = x_train[size:]
y_val = y_train[size:]

## K fold
K = 3 # nombre de fold
K_size = len(x_ent)//K # taille de chaque fold
result_fold = [] # stockage des résultats de performance de chaque fold

for i in range(K):
    x_fold_train = np.concatenate([x_ent[:i*K_size],x_ent[(i+1)*K_size:]])
    y_fold_train = np.concatenate([y_ent[:i*K_size],y_ent[(i+1)*K_size:]])

    x_fold_test = x_ent[i*K_size:(i+1)*K_size]
    y_fold_test = y_ent[i*K_size:(i+1)*K_size]
    
    model = build_model()
    model.fit(x_fold_train,
             y_fold_train,
             epochs = 5,
             batch_size = 512)
    
    validation_score = model.evaluate(x_fold_test,y_fold_test)
    result_fold.append(validation_score)
    


Epoch 1/5
25/25 [==============================] - 0s 13ms/step - loss: 0.5212 - accuracy: 0.8088
Epoch 2/5
25/25 [==============================] - 0s 14ms/step - loss: 0.2663 - accuracy: 0.9574
Epoch 3/5
25/25 [==============================] - 0s 15ms/step - loss: 0.1434 - accuracy: 0.9793
Epoch 4/5
25/25 [==============================] - 0s 14ms/step - loss: 0.0812 - accuracy: 0.9877
Epoch 5/5
196/196 [==============================] - 0s 1ms/step - loss: 0.0728 - accuracy: 0.9802
Epoch 1/5
25/25 [==============================] - 0s 14ms/step - loss: 0.5579 - accuracy: 0.7768
Epoch 2/5
25/25 [==============================] - 0s 15ms/step - loss: 0.3180 - accuracy: 0.9534
Epoch 3/5
25/25 [==============================] - 0s 15ms/step - loss: 0.1689 - accuracy: 0.9746
Epoch 4/5
25/25 [==============================] - 0s 14ms/step - loss: 0.0901 - accuracy: 0.9862
Epoch 5/5
196/196 [==============================] - 0s 1ms/step - loss: 0.0741 - accuracy: 0.9795
Epoch 1/5
25/25 [=

In [17]:
np.asarray(result_fold).mean(axis=0) # la moyenne de la perte et de l'accuracy

array([0.07322385, 0.98010667])

## Validation shuffling et kfold
- ici on va utiliser deux techniques: une randomisation des données puis kfold
- bien pour les petits jeux de données
- très computing consuming = N * K model à entrainer

In [22]:
N = 3 # nombre de shuffling à réaliser
K = 3 # nombre de fold

epoch = 5
batch_size = 512

#splitte en évaluation entrainement
p = 0.75
l = len(x_train)
size = round(p*l)

x_ent = x_train[:size]
y_ent = y_train[:size]

x_val = x_train[size:]
y_val = y_train[size:]

# boucle shuffling + boucle kfold
for n in range(N):
    indice = np.random.choice(np.arange(0,len(x_ent)),len(x_ent))
    x_ent_rand = x_ent[indice]
    y_ent_rand = y_ent[indice]
    validation_metric = np.zeros((N,K))
    k_size = len(x_train)//K
    for k in range(K):
        x_train_fold = np.concatenate([x_ent_rand[:i*k_size],x_ent_rand[(i+1)*k_size:]])
        y_train_fold = np.concatenate([y_ent_rand[:i*k_size],y_ent_rand[(i+1)*k_size:]])
        
        x_test_fold = x_ent_rand[i*k_size:(i+1)*k_size]
        y_test_fold = y_ent_rand[i*k_size:(i+1)*k_size]
        
        model = build_model()
        
        model.fit(x_train_fold,
                  y_train_fold,
                 epochs = epoch,
                 batch_size = batch_size,
                 verbose = 0)
        
        validation_metric[n,k] = model.evaluate(x_test_fold,y_test_fold)[1]
        
        
        

Epoch 1/5
33/33 [==============================] - 0s 15ms/step - loss: 0.3968 - accuracy: 0.8654
Epoch 2/5
33/33 [==============================] - 1s 16ms/step - loss: 0.1380 - accuracy: 0.9758
Epoch 3/5
33/33 [==============================] - 1s 16ms/step - loss: 0.0606 - accuracy: 0.9914
Epoch 4/5
33/33 [==============================] - 1s 17ms/step - loss: 0.0273 - accuracy: 0.9967
Epoch 5/5
66/66 [==============================] - 0s 2ms/step - loss: 0.0185 - accuracy: 0.9947
Epoch 1/5
33/33 [==============================] - 0s 15ms/step - loss: 0.4087 - accuracy: 0.8653
Epoch 2/5
33/33 [==============================] - 1s 16ms/step - loss: 0.1482 - accuracy: 0.9723
Epoch 3/5
33/33 [==============================] - 1s 15ms/step - loss: 0.0672 - accuracy: 0.9910
Epoch 4/5
33/33 [==============================] - 0s 15ms/step - loss: 0.0310 - accuracy: 0.9959
Epoch 5/5
66/66 [==============================] - 0s 1ms/step - loss: 0.0185 - accuracy: 0.9957
Epoch 1/5
33/33 [=====